In [19]:
import cv2
import tkinter as tk
from tkinter import filedialog, Scale, HORIZONTAL, Button, Label, StringVar, OptionMenu
from PIL import Image, ImageTk
import numpy as np

# Load YOLO
net = cv2.dnn.readNet("cfg/yolov3.weights", "cfg/yolov3.cfg")
with open("cfg/coco.names", "r") as f:
    classes = f.read().splitlines()
layer_names = net.getUnconnectedOutLayersNames()

# Global variables
caps = []
cap = None
paused = True
video_loaded = False
total_frames = 0
fps = 30
current_frame = 0
window = None
label = None
scale = None
play_btn = None
frame_info = None
video_selector = None
video_paths = []


def load_videos():
    global caps, video_paths, cap, current_frame, total_frames, fps, video_loaded
    file_paths = filedialog.askopenfilenames(
        filetypes=[("Video files", "*.mp4 *.avi *.mov *.mkv")]
    )
    if not file_paths:
        return
    video_paths = file_paths[:3]
    caps.clear()
    for path in video_paths:
        caps.append(cv2.VideoCapture(path))
    video_var.set(video_paths[0])
    cap = caps[0]
    configure_video()
    update_frame()


def configure_video():
    global cap, total_frames, fps, scale, current_frame, video_loaded
    if not cap:
        return
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    scale.config(to=total_frames)
    current_frame = 0
    video_loaded = True


def switch_video(*args):
    global cap, current_frame
    selected = video_var.get()
    if selected in video_paths:
        cap = caps[video_paths.index(selected)]
        configure_video()
        update_frame()


def toggle_play():
    global paused
    if not video_loaded:
        return
    paused = not paused
    play_btn.config(text="Pause" if not paused else "Play")
    if not paused:
        play_video()


def play_video():
    global current_frame, paused
    if cap.isOpened() and not paused:
        cap.set(cv2.CAP_PROP_POS_FRAMES, current_frame)
        ret, frame = cap.read()
        if not ret:
            return
        display_frame(frame)
        scale.set(current_frame)
        current_frame += 1
        if current_frame >= total_frames:
            paused = True
            play_btn.config(text="Play")
            return
        delay = int(1000 / fps)
        window.after(delay, play_video)


def display_frame(frame):
    global label, frame_info
    height, width, _ = frame.shape
    blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    outputs = net.forward(layer_names)

    boxes, confidences, class_ids = [], [], []
    for out in outputs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x, center_y = int(detection[0] * width), int(
                    detection[1] * height
                )
                w, h = int(detection[2] * width), int(detection[3] * height)
                x, y = int(center_x - w / 2), int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    for i in indexes.flatten():
        x, y, w, h = boxes[i]
        label_text = f"{classes[class_ids[i]]} {round(confidences[i], 2)}"
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(
            frame,
            label_text,
            (x, y - 10),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.6,
            (0, 255, 0),
            2,
        )

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img = Image.fromarray(rgb)
    imgtk = ImageTk.PhotoImage(image=img)
    label.imgtk = imgtk
    label.config(image=imgtk)
    frame_info.config(text=f"Frame: {current_frame}/{total_frames}")


def update_frame():
    global current_frame
    if not video_loaded:
        return
    cap.set(cv2.CAP_PROP_POS_FRAMES, current_frame)
    ret, frame = cap.read()
    if ret:
        display_frame(frame)


def seek_video(val):
    global current_frame
    if not video_loaded:
        return
    current_frame = int(val)
    update_frame()


def load_gui():
    global window, label, scale, play_btn, frame_info, video_selector, video_var
    window = tk.Tk()
    window.title("YOLO Object Detection")
    window.configure(bg="#1c1c1c")

    video_var = StringVar()

    label = tk.Label(window, bg="#1c1c1c")
    label.pack()

    scale = Scale(
        window,
        from_=0,
        to=100,
        orient=HORIZONTAL,
        length=400,
        command=seek_video,
        bg="#2c2c2c",
        fg="white",
        troughcolor="#444",
        highlightthickness=0,
    )
    scale.pack()

    frame_info = Label(window, text="Frame: 0/0", bg="#1c1c1c", fg="white")
    frame_info.pack()

    btn_frame = tk.Frame(window, bg="#1c1c1c")
    btn_frame.pack(pady=10)

    Button(
        btn_frame,
        text="Load Up to 3 Videos",
        command=load_videos,
        bg="#333",
        fg="white",
    ).pack(side=tk.LEFT, padx=5)

    play_btn = Button(
        btn_frame, text="Play", command=toggle_play, bg="#333", fg="white"
    )
    play_btn.pack(side=tk.LEFT, padx=5)

    video_selector = OptionMenu(window, video_var, "")
    video_selector.config(bg="#333", fg="white")
    video_var.trace("w", switch_video)
    video_selector.pack(pady=10)

    window.mainloop()


if __name__ == "__main__":
    load_gui()

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\testy\AppData\Local\Programs\Python\Python312\Lib\tkinter\__init__.py", line 1967, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "c:\Users\testy\AppData\Local\Programs\Python\Python312\Lib\tkinter\__init__.py", line 861, in callit
    func(*args)
  File "C:\Users\testy\AppData\Local\Temp\ipykernel_1924\181000992.py", line 84, in play_video
    display_frame(frame)
  File "C:\Users\testy\AppData\Local\Temp\ipykernel_1924\181000992.py", line 119, in display_frame
    for i in indexes.flatten():
             ^^^^^^^^^^^^^^^
AttributeError: 'tuple' object has no attribute 'flatten'
Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\testy\AppData\Local\Programs\Python\Python312\Lib\tkinter\__init__.py", line 1967, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\testy\AppData\Local\Temp\ipykernel_1924\181000992.py",

In [15]:
import cv2
import tkinter as tk
from tkinter import filedialog, Scale, HORIZONTAL, Button, Label, StringVar, OptionMenu
from PIL import Image, ImageTk
import numpy as np

# Load YOLOv3-Tiny (faster than YOLOv3)
net = cv2.dnn.readNet("cfg/yolov3-tiny.weights", "cfg/yolov3-tiny.cfg")
with open("cfg/coco.names", "r") as f:
    classes = f.read().splitlines()
layer_names = net.getUnconnectedOutLayersNames()

# Global variables
caps = []
cap = None
paused = True
video_loaded = False
total_frames = 0
fps = 30
current_frame = 0
window = None
label = None
scale = None
play_btn = None
frame_info = None
video_selector = None
video_paths = []
last_boxes = []  # Store last detection results
last_confidences = []
last_class_ids = []
detection_interval = 5  # Detect every 5 frames
frame_count = 0

def load_videos():
    global caps, video_paths, cap, current_frame, total_frames, fps, video_loaded
    file_paths = filedialog.askopenfilenames(
        filetypes=[("Video files", "*.mp4 *.avi *.mov *.mkv")]
    )
    if not file_paths:
        return
    video_paths = file_paths[:3]
    caps.clear()
    for path in video_paths:
        caps.append(cv2.VideoCapture(path))
    video_var.set(video_paths[0])
    cap = caps[0]
    configure_video()
    update_frame()

def configure_video():
    global cap, total_frames, fps, scale, current_frame, video_loaded
    if not cap:
        return
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    scale.config(to=total_frames)
    current_frame = 0
    video_loaded = True

def switch_video(*args):
    global cap, current_frame
    selected = video_var.get()
    if selected in video_paths:
        cap = caps[video_paths.index(selected)]
        configure_video()
        update_frame()

def toggle_play():
    global paused
    if not video_loaded:
        return
    paused = not paused
    play_btn.config(text="Pause" if not paused else "Play")
    if not paused:
        play_video()

def play_video():
    global current_frame, paused
    if not cap.isOpened() or paused:
        return

    # Skip frames to keep up with playback speed
    frame_skip = 2  # Adjust based on your system's performance
    for _ in range(frame_skip):
        cap.set(cv2.CAP_PROP_POS_FRAMES, current_frame)
        ret, frame = cap.read()
        if not ret:
            paused = True
            play_btn.config(text="Play")
            return
        current_frame += 1

    display_frame(frame)
    scale.set(current_frame)

    if current_frame >= total_frames:
        paused = True
        play_btn.config(text="Play")
        current_frame = 0
        return

    delay = int(1000 / (fps * frame_skip))  # Adjusted delay for frame skipping
    window.after(delay, play_video)

def display_frame(frame):
    global label, frame_info, last_boxes, last_confidences, last_class_ids, frame_count
    height, width, _ = frame.shape

    # Resize frame for faster YOLO processing
    scale_factor = 0.5  # Reduce frame size by half
    small_frame = cv2.resize(frame, (0, 0), fx=scale_factor, fy=scale_factor)
    small_height, small_width, _ = small_frame.shape

    # Perform YOLO detection every 'detection_interval' frames
    if frame_count % detection_interval == 0:
        blob = cv2.dnn.blobFromImage(small_frame, 1 / 255.0, (416, 416), swapRB=True, crop=False)
        net.setInput(blob)
        outputs = net.forward(layer_names)

        boxes, confidences, class_ids = [], [], []
        for out in outputs:
            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > 0.5:
                    center_x, center_y = int(detection[0] * small_width), int(detection[1] * small_height)
                    w, h = int(detection[2] * small_width), int(detection[3] * small_height)
                    x, y = int(center_x - w / 2), int(center_y - h / 2)
                    # Scale coordinates back to original frame size
                    x, y, w, h = [int(v / scale_factor) for v in [x, y, w, h]]
                    boxes.append([x, y, w, h])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)

        indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
        last_boxes = [boxes[i] for i in indexes.flatten()]
        last_confidences = [confidences[i] for i in indexes.flatten()]
        last_class_ids = [class_ids[i] for i in indexes.flatten()]

    frame_count += 1

    # Draw the last detected boxes
    for i, (x, y, w, h) in enumerate(last_boxes):
        label_text = f"{classes[last_class_ids[i]]} {round(last_confidences[i], 2)}"
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(
            frame,
            label_text,
            (x, y - 10),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.6,
            (0, 255, 0),
            2,
        )

    # Convert frame to RGB for display
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img = Image.fromarray(rgb)

    # Resize for faster Tkinter rendering
    max_size = (600, 400)  # Adjust based on your window size
    img.thumbnail(max_size, Image.Resampling.LANCZOS)

    imgtk = ImageTk.PhotoImage(image=img)
    label.imgtk = imgtk
    label.config(image=imgtk)
    frame_info.config(text=f"Frame: {current_frame}/{total_frames}")

def update_frame():
    global current_frame
    if not video_loaded:
        return
    cap.set(cv2.CAP_PROP_POS_FRAMES, current_frame)
    ret, frame = cap.read()
    if ret:
        display_frame(frame)

def seek_video(val):
    global current_frame
    if not video_loaded:
        return
    current_frame = int(val)
    update_frame()

def load_gui():
    global window, label, scale, play_btn, frame_info, video_selector, video_var
    window = tk.Tk()
    window.title("YOLO Object Detection")
    window.configure(bg="#1c1c1c")

    video_var = StringVar()

    label = tk.Label(window, bg="#1c1c1c")
    label.pack()

    scale = Scale(
        window,
        from_=0,
        to=100,
        orient=HORIZONTAL,
        length=400,
        command=seek_video,
        bg="#2c2c2c",
        fg="white",
        troughcolor="#444",
        highlightthickness=0,
    )
    scale.pack()

    frame_info = Label(window, text="Frame: 0/0", bg="#1c1c1c", fg="white")
    frame_info.pack()

    btn_frame = tk.Frame(window, bg="#1c1c1c")
    btn_frame.pack(pady=10)

    Button(
        btn_frame,
        text="Load Up to 3 Videos",
        command=load_videos,
        bg="#333",
        fg="white",
    ).pack(side=tk.LEFT, padx=5)

    play_btn = Button(
        btn_frame, text="Play", command=toggle_play, bg="#333", fg="white"
    )
    play_btn.pack(side=tk.LEFT, padx=5)

    video_selector = OptionMenu(window, video_var, "")
    video_selector.config(bg="#333", fg="white")
    video_var.trace("w", switch_video)
    video_selector.pack(pady=10)

    window.mainloop()

if __name__ == "__main__":
    load_gui()

error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\dnn\src\darknet\darknet_importer.cpp:210: error: (-212:Parsing error) Failed to open NetParameter file: cfg/yolov3-tiny.cfg in function 'cv::dnn::dnn4_v20231225::readNetFromDarknet'
